[View in Colaboratory](https://colab.research.google.com/github/eazhary/colab/blob/master/Predictive_CNN.ipynb)

In [0]:
!wget -q https://raw.githubusercontent.com/umbertogriffo/Predictive-Maintenance-using-LSTM/master/Dataset/PM_test.txt -O PM_test.txt 
!wget -q https://raw.githubusercontent.com/umbertogriffo/Predictive-Maintenance-using-LSTM/master/Dataset/PM_train.txt -O PM_train.txt  
!wget -q https://raw.githubusercontent.com/umbertogriffo/Predictive-Maintenance-using-LSTM/master/Dataset/PM_truth.txt -O PM_truth.txt

# In[ ]:

In [0]:
import keras
import keras.backend as K
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPool1D, GlobalMaxPooling1D, Flatten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing

In [0]:
# coding: utf-8

# In[ ]:









# Setting seed for reproducibility
np.random.seed(1234)  
PYTHONHASHSEED = 0

# define path to save model
model_path = 'regression_model.h5'


# In[ ]:


# read training data - It is the aircraft engine run-to-failure data.
train_df = pd.read_csv('PM_train.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

train_df = train_df.sort_values(['id','cycle'])

# read test data - It is the aircraft engine operating data without failure events recorded.
test_df = pd.read_csv('PM_test.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

# read ground truth data - It contains the information of true remaining cycles for each engine in the testing data.
truth_df = pd.read_csv('PM_truth.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)


# In[ ]:


train_df.describe()


# In[ ]:


##################################
# Data Preprocessing
##################################

#######
# TRAIN
#######
# Data Labeling - generate column RUL(Remaining Usefull Life or Time to Failure)
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)

# generate label columns for training data
# we will only make use of "label1" for binary classification, 
# while trying to answer the question: is a specific engine going to fail within w1 cycles?
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2

# MinMax normalization (from 0 to 1)
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
#print("Train DF", train_df.head())
#train_df.to_csv('PredictiveManteinanceEngineTraining.csv', encoding='utf-8',index = None)

######
# TEST
######
# MinMax normalization (from 0 to 1)
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
#print("Test DF",test_df.head())

# We use the ground truth dataset to generate labels for the test data.
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
#print("RUL for TEST",rul.head())
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
#print("Truth DF", truth_df.head())
truth_df.drop('more', axis=1, inplace=True)

# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)

# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
#print("Final Test",test_df.head())
#test_df.to_csv('PredictiveManteinanceEngineValidation.csv', encoding='utf-8',index = None)

# pick a large window size of 50 cycles


# In[ ]:


train_df.head()


# In[ ]:


sequence_length = 50

# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # for one id I put all the rows in a single matrix
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    # Iterate over two lists in parallel.
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191
    
    if num_elements>seq_length:
        for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
            yield data_matrix[start:stop, :]
    else:
        return
        
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

print(sequence_cols, len(sequence_cols))
# TODO for debug 
# val is a list of 192 - 50 = 142 bi-dimensional array (50 rows x 25 columns)
val=list(gen_sequence(train_df[train_df['id']==1], sequence_length, sequence_cols))
print(len(val))

print(val[0])
# generator for the sequences
# transform each id of the train dataset in a sequence
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
print(seq_array.shape)

# function to generate labels
def gen_labels(id_df, seq_length, label):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # For one id I put all the labels in a single matrix.
    # For example:
    # [[1]
    # [4]
    # [1]
    # [5]
    # [9]
    # ...
    # [200]] 
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    # I have to remove the first seq_length labels
    # because for one id the first sequence of seq_length size have as target
    # the last label (the previus ones are discarded).
    # All the next id's sequences will have associated step by step one label as target.
    if num_elements>seq_length:
        return data_matrix[seq_length:num_elements, :]
    else:
        return [None]
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
             for id in train_df['id'].unique()]

label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape


# In[ ]:


def r2_keras(y_true, y_pred):
    """Coefficient of Determination 
    """
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )


# In[ ]:


test_gen = (list(gen_sequence(test_df[test_df['id']==id], sequence_length, sequence_cols)) 
           for id in test_df['id'].unique())
test_list = list(test_gen)
#print(test_list[0:3])
test_list2= [x for x in test_list if x]
#print(test_list2[0])
# generate sequences and convert to numpy array
test_array = np.concatenate(list(test_list2)).astype(np.float32)
print(test_array.shape)

ylabel_gen = [gen_labels(test_df[test_df['id']==id], sequence_length, ['RUL']) 
             for id in test_df['id'].unique()]
yl = list(ylabel_gen)
yl = [x for x in yl if type(x) is not list]
#print(type(yl[0])
#print(yl)
ylabel_array = np.concatenate(yl).astype(np.float32)
ylabel_array.shape


# In[ ]:




['setting1', 'setting2', 'setting3', 'cycle_norm', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21'] 25
142
[[0.45977011 0.16666667 0.         ... 0.         0.71317829 0.7246617 ]
 [0.6091954  0.25       0.         ... 0.         0.66666667 0.73101353]
 [0.25287356 0.75       0.         ... 0.         0.62790698 0.62137531]
 ...
 [0.6091954  0.58333333 0.         ... 0.         0.62015504 0.65009666]
 [0.41954023 0.91666667 0.         ... 0.         0.71317829 0.76719138]
 [0.31609195 0.41666667 0.         ... 0.         0.5503876  0.71306269]]
(15631, 50, 25)
(8162, 50, 25)


(8162, 1)

In [0]:
#@title


# Next, we build a deep network. 
# The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. 
# Dropout is also applied after each LSTM layer to control overfitting. 
# Final layer is a Dense output layer with single unit and linear activation since this is a regression problem.
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()


model.add(Conv1D(filters=25,kernel_size=3,activation='relu',input_shape=(sequence_length, nb_features) ))
model.add(Dropout(0,2))
model.add(MaxPooling1D())

model.add(Conv1D(filters=25,kernel_size=3,activation='relu'))
model.add(Dropout(0,2))
model.add(MaxPooling1D())

#model.add(Conv1D(filters=50,kernel_size=3,activation='relu'))
#model.add(Dropout(0,2))
#model.add(MaxPooling1D())

model.add(Flatten())
model.add(Dropout(0,2))
model.add(Dense(1,activation='linear'))

#model.add(Conv1D(filters=100,kernel_size=2, padding='same',activation='relu', input_shape=(sequence_length, nb_features) ))
#model.add(Dropout(0,2))
#model.add(Conv1D(filters=50,kernel_size=4, padding='same',activation='relu' ))
#model.add(GlobalMaxPooling1D())
#model.add(Dense(50, activation='relu'))
#model.add(Dropout(0.2))

# We project onto a single unit output layer, and squash it with a sigmoid:
#model.add(Dense(1))
#model.add(Activation('linear'))

adam= keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='mse', optimizer=adam,metrics=['mae',r2_keras])

print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_28 (Conv1D)           (None, 48, 25)            1900      
_________________________________________________________________
dropout_32 (Dropout)         (None, 48, 25)            0         
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 24, 25)            0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 22, 25)            1900      
_________________________________________________________________
dropout_33 (Dropout)         (None, 22, 25)            0         
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 11, 25)            0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 275)               0         
__________

In [0]:
model_path='newmodel.h5'
# fit the network
history = model.fit(seq_array, label_array, epochs=100, batch_size=200, validation_split=0.05,validation_data=(test_array,ylabel_array), verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='min'),
                      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', min_delta=0, patience=3,cooldown=2, verbose=1, factor=0.5,mode='auto',min_lr=0.00001),
                       keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=1)]
          )

# list all data in history
print(history.history.keys())


Train on 15631 samples, validate on 8162 samples
Epoch 1/100
15631/15631 [==============================] - 1s 81us/step - loss: 2574.5642 - mean_absolute_error: 38.3374 - r2_keras: 0.2111 - val_loss: 3046.3493 - val_mean_absolute_error: 41.0678 - val_r2_keras: -0.6106

Epoch 00001: val_loss improved from inf to 3046.34935, saving model to newmodel.h5
Epoch 2/100
15631/15631 [==============================] - 1s 45us/step - loss: 1643.0345 - mean_absolute_error: 30.2796 - r2_keras: 0.4944 - val_loss: 1912.0434 - val_mean_absolute_error: 32.7473 - val_r2_keras: -0.0552

Epoch 00002: val_loss improved from 3046.34935 to 1912.04340, saving model to newmodel.h5
Epoch 3/100
15631/15631 [==============================] - 1s 47us/step - loss: 1538.9151 - mean_absolute_error: 28.8938 - r2_keras: 0.5271 - val_loss: 2143.6087 - val_mean_absolute_error: 39.7008 - val_r2_keras: -0.6160

Epoch 00003: val_loss did not improve from 1912.04340
Epoch 4/100
15631/15631 [==============================] -

In [0]:


# In[ ]:


# summarize history for R^2
fig_acc = plt.figure(figsize=(10, 10))
plt.plot(history.history['r2_keras'])
plt.plot(history.history['val_r2_keras'])
plt.title('model r^2')
plt.ylabel('R^2')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
fig_acc.savefig("model_r2.png")

# summarize history for MAE
fig_acc = plt.figure(figsize=(10, 10))
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('model MAE')
plt.ylabel('MAE')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
fig_acc.savefig("model_mae.png")

# summarize history for Loss
fig_acc = plt.figure(figsize=(10, 10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
fig_acc.savefig("model_regression_loss.png")

# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('\nMAE: {}'.format(scores[1]))
print('\nR^2: {}'.format(scores[2]))

y_pred = model.predict(seq_array,verbose=1, batch_size=200)
y_true = label_array

test_set = pd.DataFrame(y_pred)
test_set.to_csv('submit_train.csv', index = None)


# In[ ]:


mymodel = keras.models.load_model(model_path,custom_objects={'r2_keras': r2_keras})


# In[ ]:


scores = model.evaluate(test_array,ylabel_array, verbose=1, batch_size=200)
y_pred= model.predict(test_array)
print('\nScore 0 {}'.format(scores[0]))
print('\nMAE: {}'.format(scores[1]))
print('\nR^2: {}'.format(scores[2]))


# In[ ]:


ylabel_array.shape


# In[ ]:


import pandas as pd


# In[ ]:


df = pd.DataFrame({'truth':ylabel_array.reshape(-1), 'predict':y_pred.reshape(-1)})


# In[ ]:


df.head()


# In[ ]:


df.plot(subplots=True)



Train on 15631 samples, validate on 8162 samples
Epoch 1/100
15631/15631 [==============================] - 1s 60us/step - loss: 2512.5252 - mean_absolute_error: 38.0570 - r2_keras: 0.2295 - val_loss: 2005.2278 - val_mean_absolute_error: 36.8631 - val_r2_keras: -0.3340

Epoch 00001: val_loss improved from inf to 2005.22779, saving model to newmodel.h5
Epoch 2/100
15631/15631 [==============================] - 1s 43us/step - loss: 1506.2119 - mean_absolute_error: 28.4056 - r2_keras: 0.5358 - val_loss: 1837.4163 - val_mean_absolute_error: 35.0454 - val_r2_keras: -0.2495

Epoch 00002: val_loss improved from 2005.22779 to 1837.41626, saving model to newmodel.h5
Epoch 3/100
15631/15631 [==============================] - 1s 45us/step - loss: 1344.9566 - mean_absolute_error: 26.2887 - r2_keras: 0.5860 - val_loss: 1702.2529 - val_mean_absolute_error: 33.3105 - val_r2_keras: -0.1391

Epoch 00003: val_loss improved from 1837.41626 to 1702.25287, saving model to newmodel.h5
Epoch 4/100
15000/1563

KeyboardInterrupt: ignored